In [22]:
import os
from tqdm.notebook import tqdm
import glob
import numpy as np
import pandas as pd
import random
from collections import defaultdict
from nltk.tokenize import sent_tokenize, word_tokenize

# Read files and annotations

In [23]:
#This class is provided by i2b2 challenge. I use its functions to get annotations and text

class RecordTrack2(object):
    """Record for Track 2 class."""

    def __init__(self, file_path):
        """Initialize."""
        self.path = os.path.abspath(file_path)
        self.basename = os.path.basename(self.path)
        self.annotations = self._get_annotations()
        # self.text = self._get_text()

    @property
    def tags(self):
        return self.annotations['tags']

    @property
    def relations(self):
        return self.annotations['relations']

    def _get_annotations(self):
        """Return a dictionary with all the annotations in the .ann file."""
        annotations = defaultdict(dict)
        with open(self.path) as annotation_file:
            lines = annotation_file.readlines()
            for line_num, line in enumerate(lines):
                if line.strip().startswith('T'):
                    try:
                        tag_id, tag_m, tag_text = line.strip().split('\t')
                    except ValueError:
                        print(self.path, line)
                    if len(tag_m.split(' ')) == 3:
                        tag_type, tag_start, tag_end = tag_m.split(' ')
                    elif len(tag_m.split(' ')) == 4:
                        tag_type, tag_start, _, tag_end = tag_m.split(' ')
                    elif len(tag_m.split(' ')) == 5:
                        tag_type, tag_start, _, _, tag_end = tag_m.split(' ')
                    else:
                        print(self.path)
                        print(line)
                    tag_start, tag_end = int(tag_start), int(tag_end)
                    annotations['tags'][tag_id] = (tag_id, tag_start ,tag_end, tag_type, tag_text)
            for line_num, line in enumerate(lines):
                if line.strip().startswith('R'):
                    rel_id, rel_m = line.strip().split('\t')
                    rel_type, rel_arg1, rel_arg2 = rel_m.split(' ')
                    rel_arg1 = rel_arg1.split(':')[1]
                    rel_arg2 = rel_arg2.split(':')[1]
                    arg1 = annotations['tags'][rel_arg1]
                    arg2 = annotations['tags'][rel_arg2]
                    annotations['relations'][rel_id] = (arg1,arg2, rel_type)
        return annotations

    def get_text(self):
        """Return the text in the corresponding txt file."""
        path = self.path.replace('.ann', '.txt')
        with open(path) as text_file:
            text = text_file.read()
        return text


In [24]:
#reading data and create a list of RecordTrack2 class for each file
folder1 = 'test/'
anno = []
files1 = set([os.path.basename(f) for f in glob.glob(
    os.path.join(folder1, '*.ann'))])
for file in files1:
    anno.append(RecordTrack2(os.path.join(folder1, file)))

# Generate entities within sentence

Find the offset of each entity within that sentence. 

The offset provided by i2b2 is aginst the whole file.

In this section, we will locate the entity, tokenise sentense, and remove extra spaces

The offset I generated in this section is word offset (created by len(text.split()), feel free to change to character offset if needed. 


In [25]:
#function to generate the first entity in a sentence
def first_entity(text, start, end, entity):
        text_before = sent_tokenize(text[:start]) #all the text befere the start of the entity
        sent_num = len(text_before)
        word_before = text_before[-1].split() #words before entity in the current sentence 
        word_start = len(word_before) #start position
        word_end = word_start+len(entity.split()) #end position
        sent_after = sent_tokenize(text[end:])[0] #the left of current sentence
        sent_piece = ' '.join(word_before + entity.split()) #current sentence until the end of first entity, extra spaces removed by .split()
        return word_start, word_end, sent_after, sent_piece

In [26]:
#function to generate non-first entity in a sentence, same logic sa first_entity function
def entity_within_sent(text, word_end_before, end_before, start, end, entity):
    word_between = text[end_before:start].split()
    word_start = word_end_before + len(word_between)
    word_end = word_start+len(entity.split())
    sent_after = sent_tokenize(text[end:])[0]
    #print(sent_after)
    sent_piece = ' '.join(word_between + entity.split())
    return word_start, word_end, sent_after, sent_piece

In [27]:
tags_list = []
sents_dict = {}
for i, an in enumerate(tqdm(anno)):
    text = an.get_text()
    tags = pd.DataFrame(an.tags.values()).sort_values(1).values
    #skip used to mark how many entities we have processed within the previous sentence
    skip = 0
    for j, (en_id, start, end, concept, entity) in enumerate(tags):
        #we skip the entities we have processed from last sentence, it will bring us to the first entity in current sentence
        if skip>0:
            skip -= 1
            continue
        #remove entities that across sentences
        if len(sent_tokenize(text[start:end]))>1:
            continue
        word_start, word_end, sent_after, sent_piece = first_entity(text, start, end, entity)
        #'-'.join([str(i),str(j)]) is an unique idenfitier for each sentence in the dataset, 
        #i identify the document and j identify the sentence within the document
        tags_list.append([i, en_id, word_start, word_end, concept, entity, '-'.join([str(i),str(j)])])
        while j+skip+1<len(tags) and end+len(sent_after)>= tags[j+skip+1][1]:
            anno_curr = tags[j+skip+1]
            word_end_before = word_end
            end_before = end
            sent_piece_before = sent_piece
            en_id = anno_curr[0]
            start = anno_curr[1]
            end = anno_curr[2]
            concept = anno_curr[3]
            entity = anno_curr[4]
            word_start, word_end, sent_after, sent_piece = entity_within_sent(text, word_end_before, end_before, start, end, entity)
            sent_piece = ' '. join([sent_piece_before, sent_piece])
            tags_list.append([i, en_id, word_start, word_end, concept, entity, '-'.join([str(i),str(j)])])
            skip += 1
        sents_dict['-'.join([str(i),str(j)])] = sent_piece

  0%|          | 0/202 [00:00<?, ?it/s]

In [28]:
tags_list

[[0, 'T240', 24, 28, 'Reason', 'EtOH and cocaine abuse', '0-0'],
 [0, 'T26', 2, 3, 'Strength', '2mg', '0-1'],
 [0, 'T3', 3, 4, 'Route', 'IV', '0-1'],
 [0, 'T28', 4, 5, 'Drug', 'ativan', '0-1'],
 [0, 'T33', 6, 7, 'ADE', 'somnolence', '0-1'],
 [0, 'T35', 8, 9, 'ADE', 'apenea', '0-1'],
 [0, 'T200', 2, 3, 'Reason', 'SVT', '0-6'],
 [0, 'T214', 11, 13, 'Strength', '9 mg', '0-6'],
 [0, 'T10', 13, 14, 'Route', 'IV', '0-6'],
 [0, 'T216', 14, 15, 'Drug', 'metoprolol', '0-6'],
 [0, 'T63', 31, 32, 'Drug', 'O2', '0-10'],
 [0, 'T27', 9, 10, 'Route', 'IV', '0-11'],
 [0, 'T20', 10, 11, 'Drug', 'contrast', '0-11'],
 [0, 'T1', 22, 25, 'Reason', 'b/l subsegmental PEs', '0-13'],
 [0, 'T2', 4, 6, 'Reason', 'b/l DVTs', '0-14'],
 [0, 'T4', 4, 5, 'Drug', 'heparin', '0-15'],
 [0, 'T5', 5, 6, 'Route', 'ggt', '0-15'],
 [0, 'T6', 12, 14, 'Drug', 'anticoagulation regimen', '0-15'],
 [0, 'T7', 15, 16, 'Drug', 'coumadin', '0-15'],
 [0, 'T8', 19, 20, 'Drug', 'lovenox', '0-15'],
 [0, 'T9', 3, 4, 'Reason', 'agitated', 

In [29]:
(sents_dict)

{'0-0': 'Briefly, 47 y/o man with long standing history of epilepsy s/p right temporal lobectomy (at age 19) and VNS placement (7 years ago), h/o EtOH and cocaine abuse',
 '0-1': 'He received 2mg IV ativan with somnolence and apenea',
 '0-6': 'Patient developed SVT in the cath lab and was treated with 9 mg IV metoprolol',
 '0-10': "Family History: States has FH of diabetes and 'cancer in everyone' but no FH of CAD, HTN, sudden death or cardiac dysrhythmias Physical Exam: ON TRANSFER TO CCU: VS: HR:90 BP:128/82 O2",
 '0-11': 'Evidence of right heart strain, manifest as reflux of IV contrast',
 '0-13': '# PE/DVT: Pt noted to have enlarged RV on repeat echo [**1-31**], so CTA was obtained for concern of PEs which confirmed b/l subsegmental PEs',
 '0-14': 'LENIs also positive for b/l DVTs',
 '0-15': 'He was started on heparin ggt in house and transitioned to an anticoagulation regimen of coumadin , with a lovenox',
 '0-20': 'He was significantly agitated throughout admission which was init

In [30]:
import json
with open("test_sents_dict.txt", "w") as fp:
    json.dump(sents_dict, fp)  # encode dict into JSON

In [31]:
entity_df = pd.DataFrame(tags_list, columns=['text_id','entity_id','start','end','concept','entity','sent_num']).set_index('entity_id')

In [32]:
entity_df['concept'].unique()

array(['Reason', 'Strength', 'Route', 'Drug', 'ADE', 'Dosage',
       'Frequency', 'Form', 'Duration'], dtype=object)

In [34]:
entity_df.to_csv('test_entity.csv', sep=',', encoding='utf-8')

In [35]:
'''
This is to generate relations to a specific format for one of my model, a relation triplet is formated as 
"start_position_1st_entity, end_position_1st_entity, start_position_2nd_entity, end_position_2nd_entity,
relation_type, 1st_entity_type, 2nd_entity_type" Each relation is concatenated by all the relation triplets
within the sentence, seperated by '|'
'''

relations = []
sents = []
for i, an in enumerate(tqdm(anno)):
    text = an.get_text()
    relation = ''
    entity_curr = entity_df[entity_df['text_id']==i]#['sent_num']
    curr_sent = 0
    relation_df = pd.DataFrame.from_dict(an.relations,orient='index')#.values()
    if relation_df.empty == True:
        continue
    relation_df['head_entity'] = relation_df[1].apply(lambda x: int(x[1]))
    relation_df = relation_df.sort_values(['head_entity']).drop('head_entity',axis = 1)
    for k in relation_df.values:
        if k[0][0] not in entity_curr.index or k[1][0] not in entity_curr.index:
            continue
        e1, e2, rel = entity_curr.loc[k[0][0]], entity_curr.loc[k[1][0]], k[2]

 

        if e1['sent_num'] != e2['sent_num']:
            continue
        if curr_sent == 0:
            relation = ' '.join([str(e1['start']), str(e1['end']), str(e2['start']), str(e2['end']),
                                   '/'+rel+'/'+k[0][3]+'/'+k[1][3], '|'])
            curr_sent = e1['sent_num']
        elif e1['sent_num'] == curr_sent:           
            relation = ' '.join([relation, str(e1['start']), str(e1['end']), str(e2['start']), str(e2['end']),
                                   '/'+rel+'/'+k[0][3]+'/'+k[1][3], '|'])
        else:
            #print(curr_sent)
            relations.append((curr_sent,relation[:-2]))  
            sents.append((curr_sent,sents_dict[curr_sent]))
            curr_sent = e1['sent_num']
            relation = ' '.join([str(e1['start']), str(e1['end']), str(e2['start']), str(e2['end']),
                                   '/'+rel+'/'+k[0][3]+'/'+k[1][3],'|'])
    if curr_sent == 0:
        continue
    relations.append((curr_sent,relation[:-2]))  
    sents.append((curr_sent,sents_dict[curr_sent]))



  0%|          | 0/202 [00:00<?, ?it/s]

In [36]:
relations[0]

('0-1',
 '6 7 4 5 /ADE-Drug/ADE/Drug | 2 3 4 5 /Strength-Drug/Strength/Drug | 3 4 4 5 /Route-Drug/Route/Drug | 8 9 4 5 /ADE-Drug/ADE/Drug')

In [37]:
import re
print(re.findall('\d+', relations[0][1]))

['6', '7', '4', '5', '2', '3', '4', '5', '3', '4', '4', '5', '8', '9', '4', '5']


In [38]:
i_relation = relations[0][1].split("| ")
sample = []
for i in i_relation:
    relation_sample = []
    relation_sample.append((int(i.split(" ")[0]),int(i.split(" ")[1])))
    relation_sample.append((int(i.split(" ")[2]),int(i.split(" ")[3])))
    relation_sample.append(i.split(" ")[4].split("/")[1])
    sample.append(relation_sample)
print(sample)

[[(6, 7), (4, 5), 'ADE-Drug'], [(2, 3), (4, 5), 'Strength-Drug'], [(3, 4), (4, 5), 'Route-Drug'], [(8, 9), (4, 5), 'ADE-Drug']]


In [39]:
sents[0][1].split()

['He',
 'received',
 '2mg',
 'IV',
 'ativan',
 'with',
 'somnolence',
 'and',
 'apenea']

In [40]:
relation_data = []
for i in range(len(sents)):
    sample = []
    sample.append(sents[i][0])
    sample.append(sents[i][1].split())
    i_relation = relations[i][1].split("| ")
    i_sample = []
    for i in i_relation:
        relation_sample = []
        relation_sample.append((int(i.split(" ")[0]),int(i.split(" ")[1])))
        relation_sample.append((int(i.split(" ")[2]),int(i.split(" ")[3])))
        relation_sample.append(i.split(" ")[4].split("/")[1])
        i_sample.append(relation_sample)
    sample.append(i_sample)
    relation_data.append(sample)

In [41]:
relation_data[0]

['0-1',
 ['He',
  'received',
  '2mg',
  'IV',
  'ativan',
  'with',
  'somnolence',
  'and',
  'apenea'],
 [[(6, 7), (4, 5), 'ADE-Drug'],
  [(2, 3), (4, 5), 'Strength-Drug'],
  [(3, 4), (4, 5), 'Route-Drug'],
  [(8, 9), (4, 5), 'ADE-Drug']]]

In [21]:
import json
with open("test_relation.txt", "w") as fp:
    json.dump(relation_data, fp)